In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.4.1-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType

In [3]:
from datetime import datetime

In [4]:
#time module required to capture time and measure performance
import time 
start_time = time.time()

In [5]:
#Initiate Spark Session
spark = SparkSession \
    .builder \
    .appName("Covid Time Series Analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
#read CSV
df = spark.read.options(header=True, inferSchema = True).csv('time_series_covid19_confirmed_global.csv')

Processing Pipeline

In [7]:
#rename columns
df = df.withColumnRenamed("Country/Region", "Country")
df = df.withColumnRenamed("Province/State", "State")

#drop Columns
drop_pos_columns = ("Lat", "Long")
df = df.drop(*drop_pos_columns)

#Exclude Outliers
excluded_countries = ["Diamond Princess", "MS Zaandam", "Summer Olympics 2020", "Winter Olympics 2022"]
df = df.filter(~F.col("Country").isin(excluded_countries))

In [8]:
df.show()

+--------------------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--

In [9]:
#country Contienent mapping

country_continent_mapping = [
    ("Afghanistan", "Asia"),("Albania", "Europe"), ("Algeria", "Africa"),("Andorra", "Europe"),("Angola", "Africa"),
    ("Antarctica", "Antarctica"),("Antigua and Barbuda", "America"),("Argentina", "America"),("Armenia", "Asia"),
    ("Australia", "Oceania"),("Austria", "Europe"),("Azerbaijan", "Asia"),("Bahamas", "America"),("Bahrain", "Asia"),
    ("Bangladesh", "Asia"),("Barbados", "America"),("Belarus", "Europe"),("Belgium", "Europe"),("Belize", "America"),
    ("Benin", "Africa"),("Bhutan", "Asia"),("Bolivia", "America"),("Bosnia and Herzegovina", "Europe"),("Botswana", "Africa"),
    ("Brazil", "America"),("Brunei", "Asia"),("Bulgaria", "Europe"),("Burkina Faso", "Africa"),("Burma", "Asia"),
    ("Burundi", "Africa"),("Cabo Verde", "Africa"),("Cambodia", "Asia"),("Cameroon", "Africa"),("Canada", "America"),
    ("Central African Republic", "Africa"),("Chad", "Africa"),("Chile", "America"),("China", "Asia"),("Colombia", "America"),
    ("Comoros", "Africa"),("Congo (Brazzaville)", "Africa"),("Congo (Kinshasa)", "Africa"),("Costa Rica", "America"),
    ("Cote d'Ivoire", "Africa"),("Croatia", "Europe"),("Cuba", "America"),("Cyprus", "Asia"),("Czechia", "Europe"),
    ("Denmark", "Europe"),("Djibouti", "Africa"),("Dominica", "America"),("Dominican Republic", "America"),
    ("Ecuador", "America"), ("Egypt", "Africa"),("El Salvador", "America"),("Equatorial Guinea", "Africa"),
    ("Eritrea", "Africa"),("Estonia", "Europe"),("Eswatini", "Africa"),("Ethiopia", "Africa"),("Fiji", "Oceania"),
    ("Finland", "Europe"),("France", "Europe"),("Gabon", "Africa"),("Gambia", "Africa"),("Georgia", "Asia"),
    ("Germany", "Europe"),("Ghana", "Africa"),("Greece", "Europe"),("Grenada", "America"),("Guatemala", "America"),
    ("Guinea", "Africa"),("Guinea-Bissau", "Africa"),("Guyana", "America"),("Haiti", "America"),("Holy See", "Europe"),
    ("Honduras", "America"),("Hungary", "Europe"),("Iceland", "Europe"),("India", "Asia"),("Indonesia", "Asia"),
    ("Iran", "Asia"),("Iraq", "Asia"),("Ireland", "Europe"),("Israel", "Asia"),("Italy", "Europe"),("Jamaica", "America"),
    ("Japan", "Asia"),("Jordan", "Asia"),("Kazakhstan", "Asia"),("Kenya", "Africa"),("Kiribati", "Oceania"),
    ("Korea, North", "Asia"),("Korea, South", "Asia"),("Kosovo", "Europe"),("Kuwait", "Asia"),("Kyrgyzstan", "Asia"),
    ("Laos", "Asia"),("Latvia", "Europe"),("Lebanon", "Asia"),("Lesotho", "Africa"),("Liberia", "Africa"),
    ("Libya", "Africa"),("Liechtenstein", "Europe"),("Lithuania", "Europe"),("Luxembourg", "Europe"),("Madagascar", "Africa"),
    ("Malawi", "Africa"), ("Malaysia", "Asia"),("Maldives", "Asia"),("Mali", "Africa"),("Malta", "Europe"),("Marshall Islands", "Oceania"),
    ("Mauritania", "Africa"),("Mauritius", "Africa"),("Mexico", "America"),("Micronesia", "Oceania"),("Moldova", "Europe"),
    ("Monaco", "Europe"),("Mongolia", "Asia"),("Montenegro", "Europe"),("Morocco", "Africa"),("Mozambique", "Africa"),
    ("Namibia", "Africa"),("Nauru", "Oceania"),("Nepal", "Asia"),("Netherlands", "Europe"),("New Zealand", "Oceania"),
    ("Nicaragua", "America"),("Niger", "Africa"),("Nigeria", "Africa"),("North Macedonia", "Europe"),("Norway", "Europe"),
    ("Oman", "Asia"),("Pakistan", "Asia"),("Palau", "Oceania"),("Panama", "America"),("Papua New Guinea", "Oceania"),
    ("Paraguay", "America"),("Peru", "America"),("Philippines", "Asia"),("Poland", "Europe"),("Portugal", "Europe"),
    ("Qatar", "Asia"),("Romania", "Europe"),("Russia", "Europe"),("Rwanda", "Africa"),("Saint Kitts and Nevis", "America"),
    ("Saint Lucia", "America"),("Saint Vincent and the Grenadines", "America"),("Samoa", "Oceania"),("San Marino", "Europe"),
    ("Sao Tome and Principe", "Africa"),("Saudi Arabia", "Asia"),("Senegal", "Africa"),("Serbia", "Europe"),
    ("Seychelles", "Africa"),("Sierra Leone", "Africa"),("Singapore", "Asia"),("Slovakia", "Europe"),("Slovenia", "Europe"),
    ("Solomon Islands", "Oceania"),("Somalia", "Africa"),("South Africa", "Africa"),("South Sudan", "Africa"),
    ("Spain", "Europe"),("Sri Lanka", "Asia"),("Sudan", "Africa"),("Suriname", "America"),("Sweden", "Europe"),
    ("Switzerland", "Europe"),("Syria", "Asia"),("Taiwan*", "Asia"),("Tajikistan", "Asia"),("Tanzania", "Africa"),
    ("Thailand", "Asia"),("Timor-Leste", "Asia"),("Togo", "Africa"),("Tonga", "Oceania"),("Trinidad and Tobago", "America"),
    ("Tunisia", "Africa"),("Turkey", "Asia"),("Tuvalu", "Oceania"),("US", "America"),("Uganda", "Africa"),("Ukraine", "Europe"),
    ("United Arab Emirates", "Asia"),("United Kingdom", "Europe"),("Uruguay", "America"),("Uzbekistan", "Asia"),
    ("Vanuatu", "Oceania"),("Venezuela", "America"),("Vietnam", "Asia"),("West Bank and Gaza", "Asia"),("Yemen", "Asia"),
    ("Zambia", "Africa"),("Zimbabwe", "Africa"),
]

#converting mapping list to dataframe
continent_schema = StructType([
    StructField("Country", StringType(), True),
    StructField("Continent", StringType(), True)
])

continent_mapping_df = spark.createDataFrame(country_continent_mapping, schema=continent_schema)

continent_mapping_df.show()

+-------------------+----------+
|            Country| Continent|
+-------------------+----------+
|        Afghanistan|      Asia|
|            Albania|    Europe|
|            Algeria|    Africa|
|            Andorra|    Europe|
|             Angola|    Africa|
|         Antarctica|Antarctica|
|Antigua and Barbuda|   America|
|          Argentina|   America|
|            Armenia|      Asia|
|          Australia|   Oceania|
|            Austria|    Europe|
|         Azerbaijan|      Asia|
|            Bahamas|   America|
|            Bahrain|      Asia|
|         Bangladesh|      Asia|
|           Barbados|   America|
|            Belarus|    Europe|
|            Belgium|    Europe|
|             Belize|   America|
|              Benin|    Africa|
+-------------------+----------+
only showing top 20 rows



In [10]:
#joining contienet dataframe to main dataframe
df1 = df.join(continent_mapping_df, on=["Country"], how="left")

In [11]:
#Creating New column Zone
df2 = df1.withColumn("Zone", \
                         F.when(F.isnull(df.State), df.Country) \
                         .otherwise(df.State)
                         )

In [12]:
#pivot date and daily cases
id_vars = ["State", "Country", "Continent", "Zone"]
value_vars = [f'"{c}",`{c}`' for c in df.columns if c not in id_vars ]

stack_expr = F.expr(f"stack({len(value_vars)}, {', '.join(value_vars)}) as (date, value)")
df2_reshaped = df2.select(id_vars + [stack_expr])

#change the data type of Date column from string to date format
df2_reshaped = df2_reshaped.withColumn("date", F.to_date("date","M/d/yy"))

In [13]:
#Adding a column No_of_days that signify the total no of days elapsed from the start
min_date = df2_reshaped.select(F.min("date")).collect()[0][0]

df2_reshaped= df2_reshaped.withColumn("No_of_Days", F.datediff(F.col('date'), F.lit(min_date)))

In [14]:
#Calculate daily increases by implementing current date-previus date
    window_spec = Window.partitionBy("Zone").orderBy("date")
    df_window = df2_reshaped.orderBy("Country").withColumn("daily_increases", F.col("value") - F.lag("value").over(window_spec))

In [15]:
#dealing with the null values in daily_increases
df_window = df_window.withColumn("daily_increases", \
                         F.when(F.isnull(df_window.daily_increases), df_window.value) \
                         .otherwise(df_window.daily_increases)
                         )

# df_window1 = df_window.withColumn("date_feature", F.unix_timestamp("date"))

In [16]:
df_window.filter(df_window.Country == "China").show(2000)

+---------+-------+---------+---------+----------+-----+----------+---------------+
|    State|Country|Continent|     Zone|      date|value|No_of_Days|daily_increases|
+---------+-------+---------+---------+----------+-----+----------+---------------+
|  Beijing|  China|     Asia|  Beijing|2020-01-22|   14|         0|             14|
|  Beijing|  China|     Asia|  Beijing|2020-01-23|   22|         1|              8|
|  Beijing|  China|     Asia|  Beijing|2020-01-24|   36|         2|             14|
|  Beijing|  China|     Asia|  Beijing|2020-01-25|   41|         3|              5|
|  Beijing|  China|     Asia|  Beijing|2020-01-26|   68|         4|             27|
|  Beijing|  China|     Asia|  Beijing|2020-01-27|   80|         5|             12|
|  Beijing|  China|     Asia|  Beijing|2020-01-28|   91|         6|             11|
|  Beijing|  China|     Asia|  Beijing|2020-01-29|  111|         7|             20|
|  Beijing|  China|     Asia|  Beijing|2020-01-30|  114|         8|         

Machine Learning Tasks for Question 2

In [17]:
#import libraries
from pyspark.ml.feature import VectorAssembler #used for vector assembly, input to model
from pyspark.ml.regression import LinearRegression 

In [18]:
#Creating Vector Assembler and Linear regression class objects
vector_assembler = VectorAssembler(inputCols=["No_of_Days"], outputCol="features")

lr = LinearRegression(featuresCol='features', labelCol = 'label') 

In [19]:
#creating list of zones to iterate through
zones = df_window.select("Zone").distinct().collect()

print(zones)

[Row(Zone='Chad'), Row(Zone='Paraguay'), Row(Zone='Russia'), Row(Zone='Manitoba'), Row(Zone='Anguilla'), Row(Zone='Yemen'), Row(Zone='Korea, North'), Row(Zone='Guangdong'), Row(Zone='Senegal'), Row(Zone='Cabo Verde'), Row(Zone='Sweden'), Row(Zone='Kiribati'), Row(Zone='Hunan'), Row(Zone='Shanxi'), Row(Zone='Guyana'), Row(Zone='Burma'), Row(Zone='Eritrea'), Row(Zone='Philippines'), Row(Zone='Tibet'), Row(Zone='Jersey'), Row(Zone='Djibouti'), Row(Zone='Tonga'), Row(Zone='Malaysia'), Row(Zone='Singapore'), Row(Zone='Fiji'), Row(Zone='Hubei'), Row(Zone='Turkey'), Row(Zone='Yukon'), Row(Zone='Tianjin'), Row(Zone='Malawi'), Row(Zone='Iraq'), Row(Zone='Beijing'), Row(Zone='Northern Territory'), Row(Zone='Germany'), Row(Zone='Comoros'), Row(Zone='Afghanistan'), Row(Zone='Cambodia'), Row(Zone='Jordan'), Row(Zone='Maldives'), Row(Zone='Rwanda'), Row(Zone='Sudan'), Row(Zone='Palau'), Row(Zone='France'), Row(Zone='Turks and Caicos Islands'), Row(Zone='Greece'), Row(Zone='Kosovo'), Row(Zone='Sri La

In [20]:
result = [] #initialazing empty list

In [21]:
#iterating through each zone and calculating the trendline coefficient
for zone in zones:
    df_zone = df_window.filter(df_window['Zone'] == zone[0]).select('No_of_Days','daily_increases')
    df_zone = df_zone.withColumnRenamed('daily_increases','label')
    df_zone_vector = vector_assembler.transform(df_zone)
    model = lr.fit(df_zone_vector)
    trendline_coefficient = float(model.coefficients[0])
    #print([zone [0], trendline_coefficient])
    result.append([zone [0], trendline_coefficient])

print(result)

[['Chad', -0.006532846425561239], ['Paraguay', 0.2240196760550598], ['Russia', 16.464437211137984], ['Manitoba', 0.05721877512974767], ['Anguilla', 0.005211799384200336], ['Yemen', -0.008412905543923103], ['Korea, North', 2.1858068936684977e-06], ['Guangdong', 0.3686350147042123], ['Senegal', -0.07072891806126784], ['Cabo Verde', -0.01094125730762666], ['Sweden', 0.5866950480834147], ['Kiribati', 0.011824797419899256], ['Hunan', 0.01974093013090765], ['Shanxi', 0.024976339443981508], ['Guyana', 0.03526536643943595], ['Burma', -0.0968252183484471], ['Eritrea', -0.003273036885844923], ['Philippines', 0.40715207426805927], ['Tibet', 0.005224777612631496], ['Jersey', 0.09750150057250453], ['Djibouti', -0.022944985522725926], ['Tonga', 0.037288033385500186], ['Malaysia', 4.109667476733129], ['Singapore', 4.460320854593922], ['Fiji', 0.021016043083282452], ['Hubei', -0.28688951739408874], ['Turkey', 7.294280976090078], ['Yukon', 0.006095460037294371], ['Tianjin', 0.010284623237448089], ['Mal

Processsing Pipeline

In [22]:
#converting list to dataframe
from pyspark.sql import Row

R = Row("Zone", "trendline_coefficient")
df_result = spark.createDataFrame([R(*i) for i in result])

df_result = df_result.orderBy(F.desc("trendline_coefficient"))
df_top_100 = df_result.limit(100)

df_top_100.show(100)

+--------------------+---------------------+
|                Zone|trendline_coefficient|
+--------------------+---------------------+
|               Japan|    94.19123188154933|
|        Korea, South|    72.67960297903623|
|             Germany|    65.83472445477216|
|              France|    52.23886925252118|
|                  US|     44.6962242564584|
|               Italy|   36.660851793517246|
|             Taiwan*|    30.03070210078223|
|             Vietnam|    18.31295693610047|
|              Russia|   16.464437211137984|
|      United Kingdom|   15.647920672439058|
|              Greece|     9.71600484451953|
|             Austria|    9.637770347748694|
|     New South Wales|    8.075024743816195|
|           Hong Kong|     7.88982069086734|
|              Turkey|    7.294280976090078|
|         Netherlands|    6.953956435839991|
|               Spain|    6.472320789791172|
|            Portugal|    6.272707391527799|
|            Victoria|    5.785826797368918|
|         

In [23]:
#Joining trendline coefficient with dataframe with main dataframe and ordering it
df_top_100_stat = df_window.join(df_top_100, on = ["Zone"], how = 'inner').orderBy(['trendline_coefficient', 'date'], ascending = [False, True])

#dropping unnessary columns
drop_columns = ('Zone','State','Country','value','No of Days', 'No_of_Days', 'trendline_coefficient')
df_top_100_stat1 = df_top_100_stat.drop(*drop_columns)

#group by contienents
df_top_100_stat2 = df_top_100_stat1.groupBy("Continent",'date').agg(F.sum(df_top_100_stat.daily_increases).alias('daily_increases')).orderBy(
    ['Continent','date'], ascending = [True,True]
)

#final dataframe to be used to answer question 2
df_top_100_stat2.show()

+---------+----------+---------------+
|Continent|      date|daily_increases|
+---------+----------+---------------+
|   Africa|2020-01-22|              0|
|   Africa|2020-01-23|              0|
|   Africa|2020-01-24|              0|
|   Africa|2020-01-25|              0|
|   Africa|2020-01-26|              0|
|   Africa|2020-01-27|              0|
|   Africa|2020-01-28|              0|
|   Africa|2020-01-29|              0|
|   Africa|2020-01-30|              0|
|   Africa|2020-01-31|              0|
|   Africa|2020-02-01|              0|
|   Africa|2020-02-02|              0|
|   Africa|2020-02-03|              0|
|   Africa|2020-02-04|              0|
|   Africa|2020-02-05|              0|
|   Africa|2020-02-06|              0|
|   Africa|2020-02-07|              0|
|   Africa|2020-02-08|              0|
|   Africa|2020-02-09|              0|
|   Africa|2020-02-10|              0|
+---------+----------+---------------+
only showing top 20 rows



Cells Specific to Question 2

In [24]:
#Query to the 2nd Question
df_stat_result = df_top_100_stat2.groupBy("Continent", F.year("date").alias('Year'), F.weekofyear("date").alias("Week")).agg(
    F.mean("daily_increases").alias("weakly_mean_daily_increase"),
    F.stddev("daily_increases").alias("weakly_stddev_daily_increase"),
    F.min("daily_increases").alias("Weakly_min_daily_increase"),
    F.max("daily_increases").alias("Weakly_max_daily_increase")
).orderBy('Year','Week')

df_stat_result.show()

+---------+----+----+--------------------------+----------------------------+-------------------------+-------------------------+
|Continent|Year|Week|weakly_mean_daily_increase|weakly_stddev_daily_increase|Weakly_min_daily_increase|Weakly_max_daily_increase|
+---------+----+----+--------------------------+----------------------------+-------------------------+-------------------------+
|  America|2020|   4|                       1.6|          1.1401754250991378|                        0|                        3|
|   Africa|2020|   4|                       0.0|                         0.0|                        0|                        0|
|     Asia|2020|   4|                      51.2|          18.525657883055057|                       26|                       77|
|   Europe|2020|   4|                       0.6|          0.8944271909999159|                        0|                        2|
|  Oceania|2020|   4|                       0.8|          1.7888543819998317|             

In [25]:
#Sample results
df_stat_result.filter((df_stat_result.Year == '2020') & (df_stat_result.Week == '4')).show()

+---------+----+----+--------------------------+----------------------------+-------------------------+-------------------------+
|Continent|Year|Week|weakly_mean_daily_increase|weakly_stddev_daily_increase|Weakly_min_daily_increase|Weakly_max_daily_increase|
+---------+----+----+--------------------------+----------------------------+-------------------------+-------------------------+
|  America|2020|   4|                       1.6|          1.1401754250991378|                        0|                        3|
|   Africa|2020|   4|                       0.0|                         0.0|                        0|                        0|
|     Asia|2020|   4|                      51.2|          18.525657883055057|                       26|                       77|
|   Europe|2020|   4|                       0.6|          0.8944271909999159|                        0|                        2|
|  Oceania|2020|   4|                       0.8|          1.7888543819998317|             

In [26]:
end_time = time.time()

In [27]:
total_time = end_time-start_time

In [28]:
print(total_time)
print(start_time)
print(end_time)

3797.930243730545
1700395061.3030522
1700398859.233296


In [29]:
df_stat_result.write.csv("Answer to Question 2", header=True, mode="overwrite") 

In [30]:
df2_reshaped.printSchema()

root
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- date: date (nullable = true)
 |-- value: integer (nullable = true)
 |-- No_of_Days: integer (nullable = true)

